## Correlation Coefficient Example
The following example will calculate the correlation coefficient against 13 layered images.<br>

The Control Set is an array where each data point in the array is a spectral response at that frequency range (bandwidth) for the object of interest.  This Control Set is applied against every pixel group in the X-axis of the Data set.<br>

The layered images are stored such that:
1. X represents an image (having Y rows and Z columns)
2. Y represents the row in an image
3. Z represents the column in an image.

We start the code by loading dependencies and declare variables:

In [1]:
import numpy
import pycuda.autoinit
import pycuda.driver as cuda
from PIL import Image
from osgeo import gdal
from pycuda.gpuarray import to_gpu
from pycuda.compiler import SourceModule
from timeit import default_timer as timer

layers = 13
column = 5000
depth = 5000
layersResults = 6 #result layer and correlation formula values (5 of them)

In [2]:
#01. Open the image files,
Image00 = gdal.Open(r"Data\B01.tif")
Image01 = gdal.Open(r"Data\B02.tif")
Image02 = gdal.Open(r"Data\B03.tif")
Image03 = gdal.Open(r"Data\B04.tif")
Image04 = gdal.Open(r"Data\B05.tif")
Image05 = gdal.Open(r"Data\B06.tif")
Image06 = gdal.Open(r"Data\B07.tif")
Image07 = gdal.Open(r"Data\B08.tif")
Image08 = gdal.Open(r"Data\B08a.tif")
Image09 = gdal.Open(r"Data\B09.tif")
Image10 = gdal.Open(r"Data\B10.tif")
Image11 = gdal.Open(r"Data\B11.tif")
Image12 = gdal.Open(r"Data\B12.tif")

## Initialize our arrays
At the same time, we load dummy data where:<br>
arrayData has the same pixel value / layer<br>
arrayControl has the signature we are looking for<br>
ArrayResult has the correlation coefficient in array position [0] and [1] to [6] are the variables for the equation.<br>
Details on the equitation follow

In [3]:
arrayData = numpy.full((layers, column, depth), 0)
layers = numpy.int32(layers) #define the layer count

arrayData = arrayData.astype(numpy.int32)
#02. load them into memory
arrayData[0,:,:] = Image00.ReadAsArray()
arrayData[1,:,:] = Image01.ReadAsArray()
arrayData[2,:,:] = Image02.ReadAsArray()
arrayData[3,:,:] = Image03.ReadAsArray()
arrayData[4,:,:] = Image04.ReadAsArray()
arrayData[5,:,:] = Image05.ReadAsArray()
arrayData[6,:,:] = Image06.ReadAsArray()
arrayData[7,:,:] = Image07.ReadAsArray()
arrayData[8,:,:] = Image08.ReadAsArray()
arrayData[9,:,:] = Image09.ReadAsArray()
arrayData[10,:,:] = Image10.ReadAsArray()
arrayData[11:,:,:] = Image11.ReadAsArray()
arrayData[12:,:,:] = Image12.ReadAsArray()
    
arrayData_gpu = cuda.mem_alloc(arrayData.nbytes)
cuda.memcpy_htod(arrayData_gpu, arrayData)

arrayData_Answer = numpy.empty_like(arrayData)

arrayResult = numpy.zeros([layersResults, column, depth], dtype = numpy.float32)
arrayResult_gpu = cuda.mem_alloc(arrayResult.nbytes)
cuda.memcpy_htod(arrayResult_gpu, arrayResult)
arrayResult_Answer = numpy.empty_like(arrayResult)

arrayControl = numpy.zeros(layers, dtype = numpy.int32)

for i in range(0,13):
    arrayControl[i] = arrayData[i][0][0]
    
arrayControl_gpu = cuda.mem_alloc(arrayControl.nbytes)
cuda.memcpy_htod(arrayControl_gpu, arrayControl)

The images have been normalized in the SNAP product.<br>
The highest resolution image has a 10m resolution/pixel.<br>
The lower resolution images are stretched to match the highest resolution images by the following factors.<br>
10m / 10m = 1 -- Bands 2, 3, 4 and 8<br>
20m / 10m = 2 -- Bands 5, 6, 7, 8a, 11, 12<br>
60m / 10m = 6 -- Bands 1, 9, 10<br><br>
The following table identifies the resolution of each band for Sentinel S2A.<br>

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;border:none;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;}
.tg .tg-2ouh{background-color:#9aff99;text-align:right;vertical-align:top}
.tg .tg-l0f3{background-color:#67fd9a;text-align:center;vertical-align:top}
.tg .tg-d78e{background-color:#9aff99;text-align:center;vertical-align:top}
.tg .tg-zdbk{font-weight:bold;background-color:#003532;color:#ffffff;text-align:center;vertical-align:top}
.tg .tg-94f7{background-color:#34ff34;text-align:right;vertical-align:top}
.tg .tg-otaw{background-color:#67fd9a;text-align:right;vertical-align:top}
.tg .tg-k3lo{background-color:#34ff34;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-zdbk">Spatial Resolution<br>(m)</th>
    <th class="tg-zdbk">Band Number</th>
    <th class="tg-zdbk">Central Wavelength<br>(nm)</th>
    <th class="tg-zdbk">Bandwidth<br>(nm)</th>
  </tr>
  <tr>
    <td class="tg-d78e">10</td>
    <td class="tg-d78e">2</td>
    <td class="tg-2ouh">496.6</td>
    <td class="tg-2ouh">98</td>
  </tr>
  <tr>
    <td class="tg-d78e">10</td>
    <td class="tg-d78e">3</td>
    <td class="tg-2ouh">560.0</td>
    <td class="tg-2ouh">45</td>
  </tr>
  <tr>
    <td class="tg-d78e">10</td>
    <td class="tg-d78e">4</td>
    <td class="tg-2ouh">664.5</td>
    <td class="tg-2ouh">38</td>
  </tr>
  <tr>
    <td class="tg-d78e">10</td>
    <td class="tg-d78e">8</td>
    <td class="tg-2ouh">835.1</td>
    <td class="tg-2ouh">145</td>
  </tr>
  <tr>
    <td class="tg-l0f3">20</td>
    <td class="tg-l0f3">5</td>
    <td class="tg-otaw">703.9</td>
    <td class="tg-otaw">19</td>
  </tr>
  <tr>
    <td class="tg-l0f3">20</td>
    <td class="tg-l0f3">6</td>
    <td class="tg-otaw">740.2</td>
    <td class="tg-otaw">18</td>
  </tr>
  <tr>
    <td class="tg-l0f3">20</td>
    <td class="tg-l0f3">7</td>
    <td class="tg-otaw">782.5</td>
    <td class="tg-otaw">28</td>
  </tr>
  <tr>
    <td class="tg-l0f3">20</td>
    <td class="tg-l0f3">8a</td>
    <td class="tg-otaw">864.8</td>
    <td class="tg-otaw">33</td>
  </tr>
  <tr>
    <td class="tg-l0f3">20</td>
    <td class="tg-l0f3">11</td>
    <td class="tg-otaw">1613.7</td>
    <td class="tg-otaw">143</td>
  </tr>
  <tr>
    <td class="tg-l0f3">20</td>
    <td class="tg-l0f3">12</td>
    <td class="tg-otaw">2202.4</td>
    <td class="tg-otaw">242</td>
  </tr>
  <tr>
    <td class="tg-k3lo">60</td>
    <td class="tg-k3lo">1</td>
    <td class="tg-94f7">443.9</td>
    <td class="tg-94f7">27</td>
  </tr>
  <tr>
    <td class="tg-k3lo">60</td>
    <td class="tg-k3lo">9</td>
    <td class="tg-94f7">945.0</td>
    <td class="tg-94f7">26</td>
  </tr>
  <tr>
    <td class="tg-k3lo">60</td>
    <td class="tg-k3lo">10</td>
    <td class="tg-94f7">1373.5</td>
    <td class="tg-94f7">75</td>
  </tr>
</table>

The correlation coefficient equation is described as:<br><br>

\begin{equation*}
r = \frac{n \left(\sum XY \right) - \left( \sum X \right)\left( \sum Y \right)}
{\sqrt{\left[ n\sum X^2 - \left( \sum X\right)^2 \right]\left[ n\sum Y^2 - \left(  \sum Y\right)^2\right]}}
\end{equation*}

Using the following two arrays (these were set above during array initialization).<br>
X[ ] = {15, 18, 21, 24, 27} (Control 1D array)<br>
Y[ ] = {25, 25, 27, 31, 32} (Data where layer 1 = 25, layer 2 = 25, etc...)<br><br>
The following table calculates the values required for the formula (last row).<br> 

|ID|  X   |  Y   |  X*Y |  X*X |  Y*Y |
|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|
|Value-1|    15|    25|   375|   225|   625|
|Value-2|    18|    25|   450|   324|   625|
|Value-3|    21|    27|   567|   441|   729|
|Value-4|    24|    31|   744|   576|   961|
|Value-5|    27|    32|   864|   729|  1024|
|sum-->|105|140|3000|2295|3964|

We now substitute each value from the 'sum' row into our formula and get:<br>

\begin{equation*}
r = \frac{5 \left(3000 \right) - \left( 105 \right)\left( 140 \right)}
{\sqrt{\left[ 5\left(2295\right) - \left( 105\right)\left( 105\right) \right]\left[ 5\left( 3964\right)  - \left( 140\right)\left( 140\right)\right]}}
\end{equation*}
Thus:

\begin{equation*}
r = 0.9535
\end{equation*}

## Here we create the kernel that will be loaded into the GPU
The kernel goes through the following steps:<br>
01. define our parallel processing dimensions
02. loop through each layer and find the entries for the correlation equation
03. calculate the correlation coefficient

In [4]:
mod = SourceModule("""
    #include <math.h>
    __global__ void getLayer(int *arrayData, int *arrayControl, float *arrayResult, int layers)
    {
        //step 01
        int idx = threadIdx.x + (blockIdx.x * blockDim.x); // x coordinate (numpy axis 2) 
        int idy = threadIdx.y + (blockIdx.y * blockDim.y); // y coordinate (numpy axis 1) 
        int x_width = (blockDim.x * gridDim.x); 
        int y_width = (blockDim.y * gridDim.y);
        
        //step 02
        for(int i = 0; i < layers; i++)
        {
            //sum of X
            arrayResult[idx + (x_width * idy) + (x_width * y_width) * 1] +=
                arrayControl[i];            
            
            //sum of Y
            arrayResult[idx + (x_width * idy) + (x_width * y_width) * 2] +=
                arrayData[idx + (x_width * idy) + (x_width * y_width) * i];            
            
            //sum of X*Y
            arrayResult[idx + (x_width * idy) + (x_width * y_width) * 3] +=
                arrayData[idx + (x_width * idy) + (x_width * y_width) * i] *
                arrayControl[i];
            
            //sum of X*X
            arrayResult[idx + (x_width * idy) + (x_width * y_width) * 4] +=
                arrayControl[i] *
                arrayControl[i];            
            
            //sum of Y*Y
            arrayResult[idx + (x_width * idy) + (x_width * y_width) * 5] +=
                arrayData[idx + (x_width * idy) + (x_width * y_width) * i] *
                arrayData[idx + (x_width * idy) + (x_width * y_width) * i];            
        }
        
        //step 03
        arrayResult[idx + (x_width * idy) + (x_width * y_width) * 0] =
            fabs((layers * arrayResult[idx + (x_width * idy) + (x_width * y_width) * 3] -
               arrayResult[idx + (x_width * idy) + (x_width * y_width) * 1] *
               arrayResult[idx + (x_width * idy) + (x_width * y_width) * 2]
            )
           /
            (sqrt((layers * arrayResult[idx + (x_width * idy) + (x_width * y_width) * 4] -
                   powf(arrayResult[idx + (x_width * idy) + (x_width * y_width) * 1], 2)
                  )
                  *
                  (layers * arrayResult[idx + (x_width * idy) + (x_width * y_width) * 5] -
                   powf(arrayResult[idx + (x_width * idy) + (x_width * y_width) * 2], 2)
                  )
                 ) 
            )
           );
    }
    """)

Define the kernel in pycuda and execute the function.<br>
Once the calculation are complete, move our data from GPU memory to CPU memory.<br>
Take note: there is a timer to verify the GPU execution time.  See print-out.

In [5]:
func = mod.get_function("getLayer")
start = timer()
func(arrayData_gpu, arrayControl_gpu, arrayResult_gpu, layers, block=(20, 20, 1), grid=(250,250))
duration = timer() - start

cuda.memcpy_dtoh(arrayData_Answer, arrayData_gpu)
cuda.memcpy_dtoh(arrayResult_Answer, arrayResult_gpu)
print("It took {0:.8f} seconds to run in the GPU".format(duration))

It took 0.00107180 seconds to run in the GPU


### Save the Image

Next we save the image as 'processed.png'.<br>
And Display the unprocessed image(left) and the processed(right) image.<br>

In [6]:
rescaled = (255.0 / arrayResult_Answer[0,:,:].max() * (arrayResult_Answer[0,:,:] - arrayResult_Answer[0,:,:].min())).astype(numpy.uint8)
im = Image.fromarray(rescaled)
im.save('Data/processed.png')

|             Unprocessed             |               Processed              |
|:-----------------------------------:|:------------------------------------:|
|<img src="Data/POI1.png" width="300">|<img src="Data/delme.png" width="300">|

### Print Sample Data Results
The following displays a few data samples to demonstrate the computations performed in the GPU.<br>
In the GPU, we stored the result, along with each parameter, in arrayResult_Answer.<br>
Where:<br>
1. arrayResult_Answer[:,0] = correlation coefficient for each pixel
2. arrayResult_Answer[:,1] = \\(\sum X\\)
3. arrayResult_Answer[:,2] = \\(\sum Y\\)
4. arrayResult_Answer[:,3] = \\(\sum XY\\)
5. arrayResult_Answer[:,4] = \\(\sum XX\\)
6. arrayResult_Answer[:,5] = \\(\sum YY\\)


In [7]:
numpy.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})
print('\x1b[31;31m'+'----=== correlation coeffificient ===----'+ '\x1b[0m')
print(arrayResult_Answer[0,:,:])
print('\x1b[31;31m'+'\n----=== sum X ===----'+ '\x1b[0m')
print(arrayResult_Answer[1,:,:])
print('\x1b[31;31m'+'\n----=== sum Y ===----'+ '\x1b[0m')
print(arrayResult_Answer[2,:,:])
print('\x1b[31;31m'+'\n----=== sum XY ===----'+ '\x1b[0m')
print(arrayResult_Answer[3,:,:])
print('\x1b[31;31m'+'\n----=== sum XX ===----'+ '\x1b[0m')
print(arrayResult_Answer[4,:,:])
print('\x1b[31;31m'+'\n----=== sum YY ===----'+ '\x1b[0m')
print(arrayResult_Answer[5,:,:])

----=== correlation coeffificient ===----
[[1.0000 0.9804 0.9805 ... 0.8710 0.8737 0.8694]
 [0.9957 0.9916 0.9913 ... 0.8604 0.8615 0.8807]
 [0.9925 0.9843 0.9784 ... 0.8603 0.8610 0.8796]
 ...
 [0.8717 0.8786 0.8725 ... 0.1555 0.1538 0.1525]
 [0.8649 0.8751 0.8676 ... 0.1599 0.1570 0.1502]
 [0.8684 0.8603 0.8584 ... 0.1597 0.1599 0.1585]]

----=== sum X ===----
[[17549.0000 17549.0000 17549.0000 ... 17549.0000 17549.0000 17549.0000]
 [17549.0000 17549.0000 17549.0000 ... 17549.0000 17549.0000 17549.0000]
 [17549.0000 17549.0000 17549.0000 ... 17549.0000 17549.0000 17549.0000]
 ...
 [17549.0000 17549.0000 17549.0000 ... 17549.0000 17549.0000 17549.0000]
 [17549.0000 17549.0000 17549.0000 ... 17549.0000 17549.0000 17549.0000]
 [17549.0000 17549.0000 17549.0000 ... 17549.0000 17549.0000 17549.0000]]

----=== sum Y ===----
[[17549.0000 14775.0000 14604.0000 ... 19449.0000 19436.0000 19691.0000]
 [16599.0000 15374.0000 15436.0000 ... 19876.0000 19845.0000 19387.0000]
 [16916.0000 15789.000